# 17.06.2024 - Full patient randomization

In [ ]:
import pandas as pd
import numpy as np
import json

In [ ]:
constants = pd.read_csv("/home/makaron1/dt-gpt/uc2_nsclc/2_experiments/2024_02_05_critical_vars/1_data/constant.csv")

In [ ]:
np.random.seed(42)

# Create 2 further patient subsets
for i in range(2):

    save_path_base = "/home/makaron1/dt-gpt/uc2_nsclc/2_experiments/2024_02_05_critical_vars/1_data/patient_subsets/2024_06_17_randomized_" + str(i + 1)

    #: use instead the column dataset_split_id

    #: randomly select one dataset_split_id for validation
    validation_id = np.random.choice(constants[(constants["dataset_split_id"] != 2)]["dataset_split_id"].unique())
    validation_ids = constants[constants["dataset_split_id"] == validation_id]["patientid"].values

    #: randomly select one dataset_split_id for test which is not validation
    test_id = np.random.choice(constants[(constants["dataset_split_id"] != validation_id) & (constants["dataset_split_id"] != 1)]["dataset_split_id"].unique())
    test_ids = constants[constants["dataset_split_id"] == test_id]["patientid"].values
    
    #: select rest as training
    training_id = constants[constants["dataset_split_id"] != validation_id]
    training_id = training_id[training_id["dataset_split_id"] != test_id]
    training_ids = training_id["patientid"].values

    print("Validation ID: " + str(validation_id))
    print("Test ID: " + str(test_id))
    print("Training IDs: " + str(training_id["dataset_split_id"].unique()))

    # Assert that no overlap
    assert len(set(training_ids).intersection(set(validation_ids))) == 0
    assert len(set(training_ids).intersection(set(test_ids))) == 0
    assert len(set(validation_ids).intersection(set(test_ids))) == 0

    # Save the patient IDs as JSONs of the format { "patientids": [ "F37880F335DEF65E5E83", ...
    with open(save_path_base + "_training.json", "w") as f:
        json.dump({"patientids": training_ids.tolist()}, f)
    
    with open(save_path_base + "_validation.json", "w") as f:
        json.dump({"patientids": validation_ids.tolist()}, f)

    with open(save_path_base + "_test.json", "w") as f:
        json.dump({"patientids": test_ids.tolist()}, f)
    

    # For debug
    print(training_ids[0:10])
    print(validation_ids[0:10])
    print(test_ids[0:10])

    # Print lengths
    print(len(training_ids))
    print(len(validation_ids))
    print(len(test_ids))



